In [16]:
import os
if os.name=='nt':
    mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-8.1.0-posix-seh-rt_v6-rev0\\mingw64\\bin'
    os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sys import getsizeof
import time
import tqdm 
from matplotlib.backends.backend_pdf import PdfPages
import gc


%matplotlib inline

In [2]:
import pickle

def save_pickle(x, filename):
    with open(filename, 'wb') as handle:
        pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)

def read_pickle(filename):
    with open(filename, 'rb') as handle:
        x = pickle.load(handle)
    return x

In [3]:
feats = list(pd.read_csv('important_numeric_features.csv', index_col=0, header=None).values.ravel())
feats.insert(0, 'Id')

In [4]:
train = pd.read_csv('data/train_numeric.csv.zip', index_col=0, usecols=feats, nrows=None)
test = pd.read_csv('data/test_numeric.csv.zip', index_col=0, usecols=feats, nrows=None)
response = pd.read_csv('data/train_numeric.csv.zip', index_col=0, usecols=[0, 969], nrows=None)
n_train = len(train)

C:\Users\home\Anaconda3\envs\kaggle\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
train_test = pd.concat([train, test])
idx2id = pd.Series(train_test.index)

In [10]:
def z_score(df):
    dfz = {}
    cols = df.columns
    print('Calculate z-scores:')
    for c in tqdm.tqdm_notebook(cols):
        dfz[c+'_z'] = (df[c] - df[c].mean()) / df[c].std()
    dfz = pd.DataFrame(dfz)
    
    return dfz

In [54]:
train_test_z = z_score(train_test)

Calculate z-scores:


In [67]:
def count_samples(df):
    dfc = {}
    cols = df.columns
    print('Calculating sample counts:')
    for c in tqdm.tqdm_notebook(cols):
        dfc[c+'_c'] = df[c].map(df[c].groupby(df[c]).count())
    dfc = pd.DataFrame(dfc)
    
    return dfc

In [68]:
train_test_c = count_samples(train_test)

Calculating sample counts:


train_test['Response'] = 0
train_test.loc[response['Response']==1] = 1

In [46]:
if os.path.isfile('numeric_features.pdf'):
    os.remove('numeric_features.pdf')

with PdfPages('numeric_features.pdf') as pp:
    for f in tqdm.tqdm_notebook(train_test.columns):
        if f=='index':
            continue
        fig = plt.figure(figsize=(14, 7))
        plt.hist(train_test[f].dropna().values, bins=40, density=True)
        plt.title(f)
        plt.close(fig)
        pp.savefig(fig)

In [70]:
train_test_z = train_test_z.values
train_test_c = train_test_c.values
train_test_transformed = np.concatenate((train_test_z, train_test_c), axis=1)


In [73]:
train_transformed = train_test_transformed[:n_train, :]
test_transformed = train_test_transformed[n_train:, :]

In [ ]:
save_pickle(train_transformed, 'x_train_numeric_feat_0.pickle')
save_pickle(test_transformed, 'x_test_numeric_feat_0.pickle')